# Сетевые приложения на Python

## Сокеты

 Сокеты представляют собой программный интерфейс, который позволяет организовать взаимодействие между программами, работающими как на одном, так и на разных компьютерах в сети.

Сокеты могут использоваться для связи между программами, работающими на одном компьютере, выступая в качестве универсального механизма взаимодействия между процессами (IPC).

Чтобы установить соединение, нужно подключить модуль `socket`, создать экземпляр сокета и использовать методы для инициализации соединения, отправки и получения данных. 

### Создание объекта сокета (socket)

In [ ]:
import socket

socket.socket(socket.AF_INET, socket.SOCK_STREAM)

С помощью модуля `socket` создается объект сокета **TCP**.

**AF_INET** означает протокол адресов **IP**.

**SOCK_STREAM** означает протокол передачи **TCP**.

Комбинация **AF_INET / SOCK_STREAM** используется по умолчанию, потому что является наиболее типичной, но при этом обычно она указывается явно

### Связывание объекта с адресом (bind)

In [ ]:
import socket
host = '127.0.0.1'
port = 50007

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((host, port))

`s.bind(address)` – связывает объект сокета с адресом – для IP-адресов передается имя сервера (`host`) и номер порта (`port`).

Если попытаться открыть сокет на порту, который уже используется, Python выбросит исключение.

### Ожидание входящих запросов (listen)

In [ ]:
import socket

host = '127.0.0.1'
port = 50007

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((host, port))
    s.listen(1)

`s.listen([backlog])` – начинает ожидание входящих запросов на соединение от клиентов.

Значение, которое передаётся, определяет число клиентских запросов, которые операционная система помещает в очередь до того, как начать отклонять новые запросы (это происходит только в том случае, если сервер не справляется с обработкой запросов и очередь становится слишком длинной).

### Ожидание нового запроса на соединение (accept)

In [ ]:
with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((host, port))
    s.listen(1)
    conn, addr = s.accept()

`conn, addr = s.accept()` – ожидает получения от клиента нового запроса на подключение.

После того как он будет создан, метод `accept` вернёт новый объект сокета, через который можно обмениваться данными с подключившимся клиентом и получать данные от него.

Этот метод возвращает кортеж, состоящий из двух элементов. Первый элемент — это адрес, по которому подключился клиент. 

Метод `accept` можно вызывать несколько раз, чтобы подключить несколько клиентов. В результате каждый вызов метода возвращает новый сокет, через который происходит общение с конкретным клиентом.

### Получение данных от клиента (recv)

In [ ]:
import socket

host = '127.0.0.1'
port = 50007

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((host, port))
    s.listen(1)

    conn, addr = s.accept()

    with conn:
        print(f'Соединение установлено: {addr}')

        while True:
            data = conn.recv(1024)
            if not data:
                break

`data = conn.recv()` – принимает до 1024 байт данных, отправленных клиентом, и преобразует их в строку.

Когда клиент завершает работу, возвращается пустая строка байтов — это означает, что клиент закрыл свой конец сокета и поток данных завершился.


### Отправка сообщений (send)

In [ ]:
import socket


host = '127.0.0.1'
port = 50007

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((host, port))
    s.listen(1)

    conn, addr = s.accept()

    with conn:
        print(f'Соединение установлено: {addr}')

        while True:
            data = conn.recv(1024)
            if not data:
                break

            conn.send(data)

`conn.send()` – отправляет блок данных клиенту. 

Чтобы гарантировать надёжную передачу информации, некоторые программы могут выполнять дополнительные меры защиты. 

Например, они могут повторно отправлять недоставленные фрагменты данных или использовать метод `connection.sendall`, который принудительно отправляет все байты.

### [server] socket

In [ ]:
import socket

host = '127.0.0.1'
port = 50007


with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.bind((host, port))
    s.listen(1)
    conn, addr = s.accept()
    with conn:
        print(f'Соединение установлено: {addr}')
        while True:
            data = conn.recv(1024)
            if not data:
                break

            # Отправляем строку, закодированную в UTF-8
            conn.send('MIET-SERVER'.encode('utf-8'))

### [client] socket

In [ ]:
import socket

host = '127.0.0.1'
port = 50007

with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
    s.connect((host, port))
    s.sendall(b'Hello, world')
    data = s.recv(1024)

# Декодируем набор байт в строку с кодировкой UTF-8
print('Сервер прислал данные: ', data.decode('utf-8'))

### Модуль smtplib

Электронные письма передаются от пользователя к серверу и между серверами преимущественно по протоколу SMTP (Simple Mail Transfer Protocol). Этот протокол и его расширенная версия ESMTP описаны в стандартах RFC 821 и RFC 1869.

Для работы с SMTP в стандартной библиотеке модулей есть модуль `smtplib`. 

Чтобы начать SMTP-соединение с почтовым сервером, нужно сначала создать объект для управления SMTP-сессией с помощью конструктора класса SMTP_SSL:


```
smtplib.SMTP_SSL([host[, port]])
```

### Параметры smtplib

Параметры `host` и `port` устанавливают адрес и порт SMTP-сервера, через который будет отправляться почта. 

По умолчанию, `port=25`. Если значение `host` установлено, то конструктор самостоятельно установит соединение, иначе придется отдельно вызывать метод `connect()`.

Экземпляры класса SMTP имеют методы для всех распространенных команд SMTP-протокола, но для отправки почты достаточно вызова конструктора и методов `sendmail()` и `quit()`

### Пример отправки email сообщений

In [ ]:
from smtplib import SMTP_SSL
from email.message import EmailMessage


host = "smtp.yandex.ru"
port = 465
username = "example"
password = "qwerty"
email_from = "example@ya.ru"
email_to = "to@gmail.com"


message = EmailMessage()
message['Subject'] = 'Тема письма'
message['From'] = email_from
message['To'] = email_to
message.set_content('Текст сообщения')

with SMTP_SSL(host=host, port=port) as smtp:
    smtp.login(username, password)
    smtp.send_message(message)

### Модуль imap

Ещё один протокол — IMAP (Internet Message Access Protocol). Он позволяет пользователю эффективно работать с почтовыми ящиками на сервере.

Программа для работы с почтой, использующая этот протокол, получает доступ к почтовому хранилищу на сервере так, как если бы письма находилась на компьютере пользователя.

### Пример получения email сообщения

In [ ]:
import email
from imaplib import IMAP4_SSL


host = "imap.yandex.ru"
port = 993
username = "example"
password = "password"

with IMAP4_SSL(host, port) as M:
    rc, resp = M.login(username, password)

    # INBOX = Папка писем "Входящие"
    M.select(mailbox='INBOX', readonly=False)

    # ALL - все письма
    # UNSEEN - только непрочитанные
    typ, data = M.search(None, "UNSEEN")

    for num in data[0].split():
        typ, data = M.fetch(num, "(RFC822)")
        raw_email = data[0][1]
        message = email.message_from_bytes(raw_email)

        date = message['Date']
        email_from = message["Return-path"]
        text = message.get_payload()

        print(f"Дата: {date}")
        print(f"От кого: {email_from}")
        print(f"Содержимое письма: {text}")
